In [1]:
import pandas as pd
import dash
from dash import Dash, html, dcc, callback, Output, Input
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
from jupyter_dash import JupyterDash
import boto3
from datetime import datetime

### Loading the data 

#### (for now I'm just loading a few files here to play around with the graphs)

In [2]:
# meteo data
Q1_2022 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_2022Q1.csv')
Q2_2022 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_2022Q2.csv')
Q3_2022 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_2022Q3.csv')
Q4_2022 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_2022Q4.csv')

# combine meteo dataset 
meteocombined = pd.concat([Q1_2022, Q2_2022, Q3_2022, Q4_2022], axis=0)
meteocombined.head()

# aggregate meteo data by day
avg_meteo_combined = meteocombined.groupby(['Year','Month', 'Day']).mean()
avg_meteo_combined = avg_meteo_combined.reset_index()
avg_meteo_combined.head()

,Year,Month,Day,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,Hour,Minute,LC_RAD60,LC_TEMP_QCL0,LC_TEMP_QCL1,LC_TEMP_QCL2,LC_TEMP_QCL3
0,2022,1,1,86.557205,10.257703,36.346609,21.264531,0.000002,0.000029,-6.567639,0.322477,11.58042,25.174825,21.396634,12.447359,12.464281,12.450807,12.424740
1,2022,1,2,83.512460,9.379296,36.056435,14.140794,0.000658,0.003468,-28.866315,0.721487,11.50000,25.000000,14.099483,12.112883,12.115685,12.102359,12.086891
2,2022,1,3,89.579036,8.004425,35.882086,5.529369,0.000750,0.006831,-35.256016,0.663436,11.50000,25.000000,5.517430,9.646861,9.648249,9.635166,9.683051
3,2022,1,4,94.288450,5.983424,35.816041,7.944379,0.000469,0.003571,-25.637291,0.333304,11.50000,25.000000,7.962571,6.830167,6.828260,6.816515,6.837235
4,2022,1,5,88.026102,2.078484,35.573298,13.893690,0.000131,0.000262,-44.342911,0.609569,11.50000,25.000000,13.890103,3.903085,3.900556,3.890475,3.915760


In [3]:
# updated noise data - January
naamse35_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
naamse57_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
naamse62_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
calvarie_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
park_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
naamse81_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
kiosk_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
vrijt_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_280324_mp08bis---vrijthof.csv', header=0, sep=';')
his_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_303910_mp-04-his-hears.csv', header=0, sep=';')



In [4]:
naamse62_jan.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit
0,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:00.349,87.6,dB(A),82.7,dB(A),83.61,dB(C),97.17,dB(C)
1,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:01.349,84.5,dB(A),83.1,dB(A),84.42,dB(C),96.41,dB(C)
2,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:02.349,84.8,dB(A),82.7,dB(A),84.19,dB(C),96.24,dB(C)
3,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:03.349,81.9,dB(A),79.3,dB(A),81.08,dB(C),94.03,dB(C)
4,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:04.349,78.3,dB(A),76.0,dB(A),77.12,dB(C),89.81,dB(C)


In [ ]:
# combine noise data for January together
noise_jan_combined = pd.concat([naamse35_jan, naamse57_jan, naamse62_jan, calvarie_jan, park_jan, naamse81_jan, kiosk_jan, vrijt_jan, his_jan], axis=0)
noise_jan_combined.head()

# extract the date, month, hour, minute of "result_timestamp"
noise_jan_combined['result_timestamp'] = pd.to_datetime(noise_jan_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


noise_jan_combined['result_date'] = noise_jan_combined['result_timestamp'].dt.date
noise_jan_combined['result_month'] = noise_jan_combined['result_timestamp'].dt.month
noise_jan_combined['result_day'] = noise_jan_combined['result_timestamp'].dt.day
noise_jan_combined['result_hour'] = noise_jan_combined['result_timestamp'].dt.hour
noise_jan_combined['result_minute'] = noise_jan_combined['result_timestamp'].dt.minute

noise_jan_combined.head()

# aggregate the data by day
avg_jan_combined = noise_jan_combined.groupby(['result_date','description']).mean()
avg_jan_combined = avg_jan_combined.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
avg_jan_combined.drop(columns_to_drop, axis=1, inplace=True)
avg_jan_combined.head(150)

### Creating Plotly visualizations

### Creating a Plotly Dash app

In [2]:
# Setting up the app 
app = JupyterDash(__name__)

In [3]:
# Defining the app layout 
app.layout = html.Div(
    children=[
        html.H1("Noise Level Analysis"),
        dcc.Graph(id="noise-graph"),
        html.Label("Select a noise threshold:"),
        dcc.Slider(
            id="noise-threshold-slider",
            min=0,
            max=100,
            step=1,
            value=50,
            marks={str(i): str(i) for i in range(0, 101, 10)},
        ),
    ]
)


In [4]:
@app.callback(
    dash.dependencies.Output("noise-graph", "figure"),
    [dash.dependencies.Input("noise-threshold-slider", "value")]
)
def update_graph(noise_threshold):
    # Perform data filtering and analysis based on the noise threshold
    # Create and return a new Plotly figure
    # Example: Assume `data` is your data and `filter_data()` is a filtering function
    filtered_data = filter_data(data, noise_threshold)
    fig = create_figure(filtered_data)
    return fig


In [5]:
if __name__ == "__main__":
    app.run_server(mode='inline')


Dash is running on http://127.0.0.1:8050/



---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
NameError: name 'filter_data' is not defined

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
NameError: name 'filter_data' is not defined

